In [1]:
import tensorflow as tf
from tensorflow import keras

# 12.3. 사용자 정의 모델과 훈련 알고리즘
- 가장 간단하고 많이 사용하는 사용자 정의 손실함수를 만들어보자

### 12.3.1. 사용자 정의 손실함수
- 만약 회귀모델을 훈련하는데 훈련 세트에 잡음이 있다고 가정
- 이상치를 제거하거나 고쳐서 데이터셋을 수정해보는 방법이 가장 원론적임
    - 하지만 비효율적이고 잡음을 백프로 제거하기는 어려움
- 이럴 때는 손실함수를 바꿔서 시도하는 것도 괜춘함
    - 1) MSE : 큰 오차에 너무 큰 벌칙을 가하기 때문에 정확도가 떨어질 것
    - 2) MAE : 이상치에 너무 관대하여 최적값을 찾는 데 시간이 오래걸릴것이며 정밀도가 떨어질 것
- 이럴 때 사용하기 좋은 것이 **후버 손실**임
    - 아직 keras 공식 API지원은 없고, tf.keras.losses.Huber에서 지원하긴 함
- 이를 구현하려면 아래와 같이 레이블과 예측을 매개변수로 받는 함수를 만들고 텐서플로 연산을 이용하여 손실을 계산하면 됨

In [2]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

- 이 때 주의할 점은 **전체 손실의 평균이 아니라, 하나의 손실을 담은 텐서를 반환하는 것이 좋음**
    - 이렇게 해야 필요할 때 케라스가 클래스 가중치나 샘플 가중치를 적용할 수 있음
- 해당 손실함수를 이용하여 컴파일하는 방식은 아래와 같음

In [3]:
# model.compile(loss = huber_fn, optimizer = "nadam")
# model.fit(X_tr, y_tr, [...])

- 훈련은 이런 식으로 매우 간단하게 할 수 있음
- 하지만 모델을 저장하고 로드할 때는 문제가 발생할 수 있다!

### 12.3.2. 사용자 정의 요소를 가진 모델을 저장하고 로드하기
- 케라스가 함수 이름을 저장하므로, 사실 저장 자체에는 문제가 없음
- 하지만, 모델을 로드할 때는 함수 이름과 실제 함수를 매핑한 **딕셔너리 객체**를 전달해야함
- 좀 더 일반적으로 정리하자면, 사용자 정의 객체를 포함한 모델을 로드할 때는 그 이름과 객체를 **매핑**해줘야함
    - model을 load해올 때 custom_object 인자를 이용

In [4]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# model = keras.models.load_models("my_model.h5", custom_object = {"huber_fn" : huber_fn})

- 그런데 위와 같이 후버 함수를 만들어서 저장할 때, "작은 것"의 기준을 따로 threshold로 정하고 싶을 수 있음
- 그럴 때는 huber_fn의 함수를 정의하는 함수를 만들어 threshold를 인자로 건내주면 됨

In [5]:
def create_huber(threshold = 1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - threshold**2/2
        return tf.where(is_small_error, squared_loss, linear_loss)        
    return huber_fn

# model.compile(loss = create_huber(2.0), optimizer = "nadam")

- 그런데 이렇게 했을 때 아까처럼 모델로딩 시 문제가 또 발생하는데, threshold값이 저장되지 않기 때문임
- 그래서 모델을 로드할 때 위에서 custom_object를 잡아줬던 것 처럼 똑같이 threshold도 잡아줘야함
    - *json으로 모델 훈련시 썼던 하이퍼파라미터들을 다 저장하는 것이 좋을 것 같음...*

In [6]:
# model = keras.load_model("my_model.h5", custom_obejct   = {"huber_fn" : create_huber(2.0)})

- 참고로 새로 정의한 함수가 아닌, 컴파일 할 때 생성이 된 fn의 이름을 custom_object로 전달해야함
    - 즉 컴파일할때는 새로 정의된 함수를 썼지만, 결국 리턴된 값은 huber_fn

- 위의 문제는 keras.losses.Loss클래스를 상속하고 get_config()메서드를 구현하여 해결할 수도 있음

In [9]:
class HuberLoss(keras.losses.Loss):

    def __init__(self, threshold = 1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_samll_error, square_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold" : self.threshold}

- 이렇게하면 컴파일도 쉬워지며, 저장과정에서 threshold도 함께 저장이됨
- 대신 모델 로드 시 **클래스 이름**과 **클래스 자체**를 매핑시켜줘야함

In [10]:
# model.compile(loss = HuberLoss(2.), optimizer=  "nadam")
# model = kears.models.load_model("my_model.h5", custom_objects = {"HuberLoss" : HuberLoss})

- 모델을 저장할 떄 케라스가 손실 객체의 get_config()메서드를 호출해서 반환된 설정을 HDF5파일에 json형태로 저장
    - 그래서 모델을 로딩할 때 HuberLoss클래스의 (상속된 메서드인) from_config()클래스 메서드를 호출함
    - 이 메서드는 기본 손실 클래스에 구현되어있고 생성자에 \*\*config매개변수를 전달해서 클래스 인스턴스를 생성

### 12.3.3. 활성화 함수, 초기화, 규제, 제한 커스터마이징
- 대부분 위와 유사한 방식으로 커스터마이징 가능
    - 보통은 적절한 입/출력을 가진 간단한 함수를 작성하면 됨
- 다음은 사용자 정의 softplus활성화함수, 글로럿 초기화, L1규제, 정의 제한(양수인 가중치만 남기는 것)에 대한 예
    - 위 케이스들은 각각 keras.activations.softplus(), tf.nn.softplus()
    - keras.initializers.glorot_normal
    - keras.regularizers.l1(0.01)
    - keras.constraints.nonneg(), tf.nn.relu()와 동일함

In [11]:
def my_softplust(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

- 위에서 보듯 매개변수는 사용자가 정의하려는 함수에 따라 다름
- 